In [1]:
# To capture the NNP library stdout output we have two possible options:
#
# 1.) Install "wurlitzer" available via pip or github.com/minrk/wurlitzer
#     > pip install wurlitzer
#     and load as extension
#     > %load_ext wurlitzer
#
# 2.) Silence the library output (which will otherwise go to Jupyter server shell)
#     > nnp = pynnp.Mode()
#     > nnp.log.writeToStdout = False
#     > ... do initialization ...
#     and recall the log later:
#     > for line in nnp.log.getLog():
#     >     sys.stdout.write(line)
#
# Option 1.) is obviously more comfortable and will also capture output to stderr.
%load_ext wurlitzer
import pynnp
from sys import stdout as so
import numpy as np

In [2]:
cutoff_nl = 3.0
cutoff_HO = 2.2
mass_H = 1.00784
mass_O = 15.99903
mass_CG = mass_O + 2 * mass_H

In [3]:
%%time
structures = []
for s in pynnp.DatasetReader("/bdata/projects/cgnnp/02_analyze-descriptors/cg-h2o-com/atomistic-description/input.data", "H O"):
    s.calculateNeighborList(cutoff_nl)
    break

CPU times: user 6.45 s, sys: 17.2 ms, total: 6.47 s
Wall time: 6.44 s


In [4]:
a = s.atoms[0]

In [7]:
a.index = False

In [10]:
a.hasSymmetryFunctions

True

In [6]:
ra = s.atoms[0].r
print(s.atoms[0].r)

x: 39.551165 y: 41.460701 z: 139.067578


In [6]:
a = s.atoms[0]
print(a)

In [12]:
print(a.neighbors[0].dr)

x: 4.000000 y: 5.000000 z: 6.000000


In [25]:
dr = a.neighbors[0].dr

In [4]:
v = pynnp.Vec3D(1, 2, 3)

In [11]:
s.atoms[0].neighbors[0].dr = pynnp.Vec3D(4, 5, 6)

In [9]:
n = a.neighbors[0]

In [10]:
n.dGdr

[]

In [34]:
n.dr = v

AttributeError: attribute 'dr' of 'pynnp.Neighbor' objects is not writable

In [28]:
dr = v

In [5]:
s.atoms[0].r = pynnp.Vec3D(1, 2, 3)
s.atoms[0].r.r

[1.0, 2.0, 3.0]

In [6]:
r1 = pynnp.Vec3D(1, 2, 3)
r1

In [7]:
a.r = r1

In [8]:
a.r

[1.0, 2.0, 3.0]

In [9]:
r2 = a.r

In [13]:
b = pynnp.Vec3D(True)

In [ ]:
b.r

In [6]:
oxygens = []
for atom in s.atoms:
    if s.elementMap[atom.element] == "O":
        oxygens.append(atom.index)
    # Set hydrogen tag to zero for later check.
    atom.tag = 0
print(len(oxygens))

2304


Center-of-mass coordinates:

$$\vec{R}_i = \frac{\sum_j m_j \vec{r}_j}{\sum_j m_j} = \frac{\sum_j m_j \vec{r}_j}{M}$$

Consider only a single water molecule

$$\vec{R} = \frac{m_O \vec{r}_O + m_H \left( \vec{r}_{H_1} + \vec{r}_{H_2} \right)}{m_O + 2 m_H}$$

Use $\Delta \vec{r}_{jk} = \vec{r}_j - \vec{r}_k$

$$\Delta \vec{r}_{OH_i} = \vec{r}_{O} - \vec{r}_{H_i} \Rightarrow \vec{r}_{H_i} = \vec{r}_{O} - \Delta \vec{r}_{OH_i}$$

gives

$$
\begin{align}
\vec{R} & = \frac{m_O \vec{r}_O + m_H \left(\vec{r}_{O} - \Delta \vec{r}_{OH_1} + \vec{r}_{O} - \Delta \vec{r}_{OH_2} \right)}{m_O + 2 m_H} \\
& = \frac{m_O \vec{r}_O + 2 m_H \vec{r}_{O} - 2 m_H \left(\Delta \vec{r}_{OH_1} + \Delta \vec{r}_{OH_2} \right)}{m_O + 2 m_H} \\
& = \vec{r}_O - \frac{2 m_H \left(\Delta \vec{r}_{OH_1} + \Delta \vec{r}_{OH_2} \right)}{m_O + 2 m_H}
\end{align}
$$

Center-of-mass force:

$$\vec{F}_i = M \sum_j m_j \vec{r}_j = \sum \vec{f}_j$$

In [ ]:
%load_ext line_profiler

In [ ]:
def create_cg_structure():
    s_cg = pynnp.Structure()
    em_cg = pynnp.ElementMap()
    em_cg.registerElements("Cm")
    s_cg.setElementMap(em_cg)

    atoms = s.atoms
    for o in oxygens:
        nl = [n for n in atoms[o].neighbors]
        nl.sort()
        n_H = [n for n in nl if n.d < cutoff_HO and s.elementMap[n.element] == "H"]
        if len(n_H) != 2:
            raise RuntimeError("ERROR: Could not find two hydrogen neighbors!")
        atom_cg = pynnp.Atom()
        atom_cg.r.r = atoms[o].r - 2 * mass_H * (n_H[0].dr + n_H[1].dr) / mass_CG
        atom_cg.fRef.r = atoms[o].fRef + atoms[n_H[0].index].fRef + atoms[n_H[1].index].fRef
        s_cg.addAtom(atom_cg, "Cm")
    return s_cg

In [ ]:
%time s_cg = create_cg_structure()

In [ ]:
atoms = s_cg.atoms

In [ ]:
a0 = atoms[0]
a1 = atoms[1]
a2 = atoms[2]

In [ ]:
print(a2.r)
print(a2.r.__repr__())

In [ ]:
d = a2.r

In [ ]:
d[0] = -2

In [ ]:
print(a2.r)
print(a2.r.__repr__())

In [ ]:
b = a0.r + a1.r

In [ ]:
b

In [ ]:
c = pynnp.Vec3D(b)

In [ ]:
c.r

In [ ]:
print(a2.r)
print(a2.r.__repr__())

In [ ]:
print(b)
print(b.__repr__())

In [ ]:
print(c)
print(c.__repr__())

In [ ]:
c = b
c[0] = -3

In [ ]:
print(c)
print(c.__repr__())

In [ ]:
print(b)
print(b.__repr__())

In [ ]:
a2.info()